# Changing the format of data in label's file from ms to timedate

In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
df= pd.read_csv('labels.csv')

In [3]:
df.head()

,Unnamed: 0,user,begin,end
0,0,patient_Jonas,1623622759524,1623652279524
1,1,patient_Jonas,1624406119524,1624432399524
2,2,patient_Jonas,1623888439524,1623906019524
3,3,patient_Jonas,1624229359524,1624255819524
4,4,patient_Jonas,1624314619524,1624347619524


In [4]:
dtype_before1 = type(df['begin'])
dtype_before1 = type(df['end'])

In [5]:
beging_list = df['begin'].tolist()
end_list = df['end'].tolist()

In [6]:
type(beging_list)

list

In [7]:
# Changing the format from ms to datetime



begin_after=[]
end_after=[]

for x in beging_list:
    begin_after.append(
    datetime.datetime.fromtimestamp(
        int(x)*0.001
    ).strftime('%Y-%m-%d %H:%M:%S'))

    


for y in end_list:
    end_after.append(
    datetime.datetime.fromtimestamp(
        int(y)*0.001
    ).strftime('%Y-%m-%d %H:%M:%S'))



In [8]:
print(begin_after)
print(end_after)

['2021-06-14 00:19:19', '2021-06-23 01:55:19', '2021-06-17 02:07:19', '2021-06-21 00:49:19', '2021-06-22 00:30:19', '2021-06-17 23:17:05', '2021-06-10 00:53:05', '2021-06-08 23:23:05', '2021-06-16 23:53:05', '2021-06-09 01:37:05', '2021-06-14 04:00:05', '2021-06-16 20:00:05', '2021-06-13 00:30:05', '2021-06-16 00:00:05', '2021-06-14 22:31:05', '2021-06-06 23:30:05', '2021-06-08 02:07:05', '2021-06-09 05:59:48', '2021-06-09 00:05:48', '2021-06-08 02:29:48', '2021-06-16 23:14:28', '2021-06-21 23:44:35', '2021-06-10 00:40:03', '2021-06-08 01:52:03', '2021-06-09 01:46:03', '2021-06-15 20:06:08', '2021-06-19 05:42:08', '2021-06-19 00:45:07', '2021-06-16 22:54:08', '2021-06-20 00:32:14', '2021-06-20 00:17:07', '2021-06-16 22:48:08', '2021-06-20 02:42:14']
['2021-06-14 08:31:19', '2021-06-23 09:13:19', '2021-06-17 07:00:19', '2021-06-21 08:10:19', '2021-06-22 09:40:19', '2021-06-18 06:41:05', '2021-06-10 06:35:05', '2021-06-09 06:41:05', '2021-06-17 06:47:05', '2021-06-09 06:55:05', '2021-06-

In [9]:
user=df['user']

In [10]:
d= {'user':user, 'begin':begin_after, 'end':end_after}

In [11]:
df_changed_format= pd.DataFrame(d, columns=['user', 'begin','end'])

In [12]:
df_changed_format.head()

,user,begin,end
0,patient_Jonas,2021-06-14 00:19:19,2021-06-14 08:31:19
1,patient_Jonas,2021-06-23 01:55:19,2021-06-23 09:13:19
2,patient_Jonas,2021-06-17 02:07:19,2021-06-17 07:00:19
3,patient_Jonas,2021-06-21 00:49:19,2021-06-21 08:10:19
4,patient_Jonas,2021-06-22 00:30:19,2021-06-22 09:40:19


In [13]:
df_changed_format.to_csv(r'Changed_format_labels1.csv')

In [14]:
df2= pd.read_csv('Changed_format_labels1.csv')

In [15]:
df2.head()

,Unnamed: 0,user,begin,end
0,0,patient_Jonas,2021-06-14 00:19:19,2021-06-14 08:31:19
1,1,patient_Jonas,2021-06-23 01:55:19,2021-06-23 09:13:19
2,2,patient_Jonas,2021-06-17 02:07:19,2021-06-17 07:00:19
3,3,patient_Jonas,2021-06-21 00:49:19,2021-06-21 08:10:19
4,4,patient_Jonas,2021-06-22 00:30:19,2021-06-22 09:40:19


# Labeling for all data

In [21]:
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore') 
import shutil

In [17]:
# Aim: building a table to recognize the amount of data recorded of each feature for each subject 
df_transfer_label=pd.read_csv('Changed_format_labels.csv')

f=pd.read_csv('features.csv')
features_name=list(f['features'])

n=pd.read_csv('names.csv')
subjects_name=list(n['names'])

In [18]:
# Building the table

init=np.zeros((len(subjects_name),len(features_name)))
summary = pd.DataFrame (init, columns = features_name,index=subjects_name)



# filling the table with the number of recorded data while user was sleeping

# finding the during of sleep time
directory='data'
for i in os.listdir(directory):
    if i[-3:]=='csv':
        df=pd.read_csv(directory+'/'+i)
        name=i.split('_')[0]
        feature=i.split('_')[1]
        
        begin_time=list(df_transfer_label['begin'][(df_transfer_label['user']==name)])
        end_time=list(df_transfer_label['end'][(df_transfer_label['user']==name)])
        
        # assumption of datetime format: for having easier the procedure 
        # and the same format like changed format datetime
        split_time=[]
        for x in df['timestamp']:
            split_time.append(x.split()[0] +' '+ x.split()[1][:8])
            
        df['time_new']=split_time
        
        
        # Adding a label if user was sleeping or not ## Sleep=1 Up=0
        df['Label']=np.zeros((len(df),1))
        for i in range(len(begin_time)):
            mask=(df['time_new'] > begin_time[i])&(df['time_new'] <= end_time[i])
            df['Label'][mask]=1
        
        df.to_csv('labeled_'+name+'_'+feature+'_'+'data.csv')
        
        summary[feature][name]=sum(df['Label']==1)
        
        
        

In [19]:
number_sleep= df_transfer_label['user'].value_counts()
summary['number of sleep']=number_sleep

summary

,light,noise,proximity,screen,step,number of sleep
Jonas,56043.0,2086.0,3.0,4.0,0.0,5
svhoecke,49028.0,0.0,0.0,6.0,17.0,4
fongenae,142279.0,0.0,94.0,93.0,0.0,8
Emiel,0.0,14.0,0.0,0.0,0.0,3
Nicolas,26459.0,0.0,4.0,6.0,0.0,1
Diego,19934.0,0.0,2.0,0.0,0.0,1
Jessica,0.0,12351.0,7.0,0.0,0.0,3
Celine,99369.0,0.0,1.0,1.0,0.0,8


# Survay on features extracted by table

# light

In [30]:
# light- calculation  of mean and std 

df1=pd.read_csv('labeled_data/labeled_Jonas_light_data.csv')
df2=pd.read_csv('labeled_data/labeled_fongenae_light_data.csv')
df3=pd.read_csv('labeled_data/labeled_Celine_light_data.csv')
df4=pd.read_csv('labeled_data/labeled_Diego_light_data.csv')
df5=pd.read_csv('labeled_data/labeled_Nicolas_light_data.csv')
df6=pd.read_csv('labeled_data/labeled_svhoecke_light_data.csv')
df=pd.concat([df1,df2,df3,df4,df5,df6])


init=np.zeros((2,2))
init[0,0]= np.mean(df['lux'][(df['Label']==1)])
init[0,1]= np.mean(df['lux'][(df['Label']==0)])

init[1,0]= np.std(df['lux'][(df['Label']==1)])
init[1,1]= np.std(df['lux'][(df['Label']==0)])


summary_table=pd.DataFrame(init, columns=['sleep','up'], index=['mean','std'])
summary_table

FileNotFoundError: [Errno 2] No such file or directory: 'labeled_data/labeled_Jonas_light_data.csv'

In [ ]:
sns.displot(df,x='lux',kind="kde",hue='Label',palette='magma')

# noise

In [ ]:
df=pd.read_csv('labeled_data/labeled_Jonas_noise_data.csv')



init1=np.zeros((2,2))
init1[0,0]=np.mean(df['frequency'][(df['Label']==1)])
init1[0,1]=np.mean(df['frequency'][(df['Label']==0)])

init1[1,0]=np.std(df['frequency'][(df['Label']==1)])
init1[1,1]=np.std(df['frequency'][(df['Label']==0)])
summary_table1 = pd.DataFrame (init1, columns = ['sleep','sp'],index=['mean','std'])



init2=np.zeros((2,2))
init2[0,0]=np.mean(df['amplitude'][(df['Label']==1)])
init2[0,1]=np.mean(df['amplitude'][(df['Label']==0)])
init2[1,0]=np.std(df['amplitude'][(df['Label']==1)])
init2[1,1]=np.std(df['amplitude'][(df['Label']==0)])
summary_table2 = pd.DataFrame (init2, columns = ['sleep','up'],index=['mean','std'])


frequency

In [ ]:
summary_table1

amplitude

In [ ]:
summary_table2

In [ ]:

sns.scatterplot(x='frequency',y='amplitude',data=df,hue='Label',palette='magma',
           markers=['o','v'])

In [ ]:
sns.displot(df,x='frequency',kind="kde",hue='Label',palette='magma')

In [ ]:
df_counts = (df.groupby(['Label'])['amplitude']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('amplitude'))


sns.displot(df_counts,x='amplitude',kind="kde",
            hue='Label',palette='magma')

# proximity

In [ ]:
df=pd.read_csv('labeled_data/labeled_fongenae_proximity_data.csv')
df.head()

In [ ]:
init=np.zeros((2,2))

init[0,0]=np.mean(df['proximity'][(df['Label']==1)])
init[0,1]=np.mean(df['proximity'][(df['Label']==0)])

init[1,0]=np.std(df['proximity'][(df['Label']==1)])
init[1,1]=np.std(df['proximity'][(df['Label']==0)])

summary_table3 = pd.DataFrame (init, columns = ['sleep','up'],index=['mean','std'])

summary_table3

In [ ]:
summary_table3.iplot(kind='bar')

# screen

In [ ]:
df=pd.read_csv('labeled_data/labeled_fongenae_screen_data.csv')
df.head()

In [ ]:
init=np.zeros((2,4))

n_sleep=sum(df['Label']==1)
n_up=sum(df['Label']==0)

init[0,0]=(sum((df['Label']==1)&(df['screen_state']=='On')))/(n_sleep)
init[0,1]=sum((df['Label']==1)&(df['screen_state']=='Off'))/(n_sleep)
init[0,2]=sum((df['Label']==1)&(df['screen_state']=='Locked'))/(n_sleep)
init[0,3]=sum((df['Label']==1)&(df['screen_state']=='Unlocked'))/(n_sleep)


init[1,0]=(sum((df['Label']==0)&(df['screen_state']=='On')))/(n_up)
init[1,1]=sum((df['Label']==0)&(df['screen_state']=='Off'))/(n_up)
init[1,2]=sum((df['Label']==0)&(df['screen_state']=='Locked'))/(n_up)
init[1,3]=sum((df['Label']==0)&(df['screen_state']=='Unlocked'))/(n_up)


summary_table4 = pd.DataFrame (init, columns = ['On', 'Off', 'Locked','Unlocked'],index=['sleep','up'])

summary_table4

In [ ]:
summary_table4.iplot(kind='bar')

# step

In [ ]:
df=pd.read_csv('labeled_data/labeled_svhoecke_step_data.csv')
df.head()

In [ ]:
df['steps'].unique()

In [ ]:
sum(df['steps'])

In [ ]:
sns.distplot(df['steps'])

# Machine learning model

In [ ]:
df=pd.read_csv('labeled_data/labeled_Jonas_noise_data.csv')

In [ ]:
print('Number of sleep data '+str(sum(df['Label']==1)))

print('Number of up data '+str(sum(df['Label']==0)))

In [ ]:
sleep_data=df[['frequency','amplitude']].loc[df['Label']==1]
sleep_data=sleep_data.to_numpy()

up_data=df[['frequency','amplitude']].loc[df['Label']==0]
up_data=up_data.to_numpy()

In [ ]:
#for avoiding the imbalance data, we added some of random samples from up_data to sleep_data

In [ ]:
type(sleep_data)

In [ ]:
sleep_data=sleep_data[:2085, : ] 
up_data=up_data[:2085, : ] 

In [ ]:
lenght_segment= 5 #assume i.e. minutes 


n_segment=int(2085/5)

feat_sleep=np.zeros((n_segment,4))
feat_up=np.zeros((n_segment, 4))


for i in range (n_segment):
    p= sleep_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    n= up_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    
    
    feat_sleep[i,0:2]=np.mean(p, axis=0)
    feat_sleep[i, 2:]=np.std(p, axis=0)
    
    
    feat_up[i,0:2]= np.mean(n, axis=0)
    feat_up[i,2:]= np.std(n, axis=0)


x= np.concatenate([feat_sleep,feat_up], axis=0)
y= np.zeros(((n_segment)*2,1))

y[:n_segment,0]=1

In [ ]:
y_new= np.reshape(y[:,0], newshape=(y.shape[0],1))
x_new= np.concatenate([x,y_new],1)
df_feat=pd.DataFrame(data=x_new,columns=['mean_fre','mean_amp','std_fre','std_amp','label'])

In [ ]:
df_feat.head()

In [ ]:
sns.pairplot(df_feat,hue='label',palette='magma')

# SVM 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import cmocean

In [ ]:
X_train , X_test , y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
clf = svm.SVC()
clf.fit(X_train, y_train[:,0])
y_pred=clf.predict(X_test)
conf=confusion_matrix(y_test, y_pred)
conf=conf/np.sum(conf,axis=1)

In [ ]:
plot_confusion_matrix(clf, X_test, y_test[:,0],cmap='GnBu',normalize='true')  
plt.show()  

In [ ]:
from sklearn.metrics import classification_report
y_actu = pd.Series(np.reshape(y_test,newshape=(y_test.shape[0],)), name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
target_names=['sleep','up']
print(classification_report(y_actu, y_pred, target_names=target_names))

In [ ]:
y_test=np.reshape(y_test,newshape=(y_test.shape[0],))
y_test.shape

# Subject wise prediction

In [ ]:
df=pd.read_csv('labeled_data/labeled_Jessica_noise_data.csv')
df.head()

In [ ]:
print('Number of sleep data '+str(sum(df['Label']==1)))

print('Number of up data '+str(sum(df['Label']==0)))

In [ ]:
sleep_data=df[['frequency','amplitude',]].loc[(df['Label']==1)]
sleep_data=sleep_data.to_numpy()

up_data=df[['frequency','amplitude',]].loc[(df['Label']==0)]
up_data=up_data.to_numpy()

In [ ]:
sleep_data=sleep_data[:2085, : ] 
up_data=up_data[:2085, : ] 


In [ ]:
lenght_segment= 5 #mins

n_segment=int(2085/5)
feat_sleep=np.zeros((n_segment,4))
feat_up=np.zeros((n_segment,4))


for i in range (n_segment):
    
    p=sleep_data[i*lenght_segment:(i+1)*lenght_segment, : ]
    n=up_data[i*lenght_segment:(i+1)*lenght_segment, : ]
    
    feat_sleep[i,0:2]= np.mean(p, axis=0)
    feat_sleep[i,2:]= np.std(p, axis=0)
    
    feat_up[i,0:2]= np.mean(n, axis=0)
    feat_up[i,2:]= np.std(n, axis=0)
    

x1=np.concatenate([feat_sleep,feat_up], axis=0)
y=np.zeros((n_segment*2,1))
y[:n_segment,0]=1


In [ ]:
plot_confusion_matrix(clf, x, y,cmap='GnBu',normalize='true',labels=['sleep','up'])  
plt.show()  

In [ ]:
y_actu = pd.Series(ytrue, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
target_names=['sleep','up']
print(classification_report(y_actu, y_pred, target_names=target_names))

# Light- Machine learning model

In [ ]:
dff=pd.read_csv('labeled_data/labeled_fongenae_light_data.csv')
dfj=pd.read_csv('labeled_data/labeled_jonas_light_data.csv')
df_light=pd.concat([dff,dfj])

In [ ]:
df_light.head()

In [ ]:
print('Number of sleep data '+str(sum(df_light['Label']==1)))

print('Number of up data '+str(sum(df_light['Label']==0)))

In [ ]:
sleep_data=df_light[['lux']].loc[df_light['Label']==1]
sleep_data=sleep_data.to_numpy()

up_data=df_light[['lux']].loc[df_light['Label']==0]
up_data=up_data.to_numpy()

In [ ]:
sleep_data=sleep_data[:198320, : ] 
up_data=up_data[:198320, : ] 

In [ ]:
lenght_segment= 5 #assume i.e. minutes 


n_segment=int(198320/5)

feat_sleep=np.zeros((n_segment,2))
feat_up=np.zeros((n_segment, 2))


for i in range (n_segment):
    p= sleep_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    n= up_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    
    
    feat_sleep[i,0:1]=np.mean(p, axis=0)
    feat_sleep[i, 1:]=np.std(p, axis=0)
    
    
    feat_up[i,0:1]= np.mean(n, axis=0)
    feat_up[i,1:]= np.std(n, axis=0)


x= np.concatenate([feat_sleep,feat_up], axis=0)
y= np.zeros(((n_segment)*2,1))

y[:n_segment,0]=1

In [ ]:
y_new= np.reshape(y[:,0], newshape=(y.shape[0],1))
x_new= np.concatenate([x,y_new],1)
df_feat=pd.DataFrame(data=x_new,columns=['mean_lux','std_lux','label'])

In [ ]:
df_feat

In [ ]:
198320/5

In [ ]:
sns.pairplot(df_feat,hue='label',palette='magma')

In [ ]:
X_train , X_test , y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
clf = svm.SVC()
clf.fit(X_train, y_train[:,0])
y_pred=clf.predict(X_test)
conf=confusion_matrix(y_test, y_pred)
conf=conf/np.sum(conf,axis=1)

In [ ]:
plot_confusion_matrix(clf, X_test, y_test[:,0],cmap='GnBu',normalize='true')  
plt.show()

# Subject-wise light

In [ ]:
df=pd.read_csv('labeled_data/labeled_svhoecke_light_data.csv')
df.head()

In [ ]:
print('Number of sleep data '+str(sum(df['Label']==1)))

print('Number of up data '+str(sum(df['Label']==0)))

In [ ]:
sleep_data=df[['lux']].loc[(df['Label']==1)]
sleep_data=sleep_data.to_numpy()

up_data=df[['lux']].loc[(df['Label']==0)]
up_data=up_data.to_numpy()

In [ ]:
sleep_data=sleep_data[:49020, : ] 
up_data=up_data[:49020, : ] 

In [ ]:
lenght_segment= 5 #assume i.e. minutes 


n_segment=int(49020/5)

feat_sleep=np.zeros((n_segment,2))
feat_up=np.zeros((n_segment, 2))


for i in range (n_segment):
    p= sleep_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    n= up_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    
    
    feat_sleep[i,0:1]=np.mean(p, axis=0)
    feat_sleep[i, 1:]=np.std(p, axis=0)
    
    
    feat_up[i,0:1]= np.mean(n, axis=0)
    feat_up[i,1:]= np.std(n, axis=0)


x= np.concatenate([feat_sleep,feat_up], axis=0)
y= np.zeros(((n_segment)*2,1))

y[:n_segment,0]=1

In [ ]:
plot_confusion_matrix(clf, x, y,cmap='GnBu',normalize='true')  
plt.show()  

# screen- Machine learning model

In [ ]:
df_screen=pd.read_csv('labeled_data/labeled_fongenae_screen_data.csv')

df_screen.head()

In [ ]:
print('Number of sleep data '+str(sum(df_screen['Label']==1)))

print('Number of up data '+str(sum(df_screen['Label']==0)))

In [ ]:
a=df_screen.loc[df_screen['screen_state']=='Unlocked']


In [ ]:
sleep_data=df_screen.loc[df_screen['screen_state']=='Unlocked'].loc[df_screen['Label']==1]
sleep_data=sleep_data.to_numpy()

up_data=df_screen.loc[df_screen['screen_state']=='Unlocked'].loc[df_screen['Label']==0]
up_data=up_data.to_numpy()

In [ ]:
sleep_data=sleep_data[:90, : ] 
up_data=up_data[:90, : ] 

In [ ]:
lenght_segment= 5 #assume i.e. minutes 


n_segment=int(90/5)

feat_sleep=np.zeros((n_segment,2))
feat_up=np.zeros((n_segment, 2))


for i in range (n_segment):
    p= sleep_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    n= up_data[i*(lenght_segment):(i+1)*(lenght_segment), :]
    
    
    feat_sleep[i,0:1]=np.mean(p, axis=0)
    feat_sleep[i, 1:]=np.std(p, axis=0)
    
    
    feat_up[i,0:1]= np.mean(n, axis=0)
    feat_up[i,1:]= np.std(n, axis=0)


x= np.concatenate([feat_sleep,feat_up], axis=0)
y= np.zeros(((n_segment)*2,1))

y[:n_segment,0]=1